# Multilabel model
Finding the best model for classification of multiple emotions for tekst messages. Using LSTM, CNN and Transformer models. \\
Multi-Class Emotion Detection Model using Deep Learning

This implementation builds and compares multiple neural network architectures
for multi-label emotion classification using the GoEmotions dataset.

Main references:
[1] Demszky, D., Movshovitz-Attias, D., Ko, J., Cowen, A., Nemade, G., & Ravi, S. (2020).\\
    "GoEmotions: A Dataset of Fine-Grained Emotions." arXiv preprint arXiv:2005.00547.\\
    https://github.com/google-research/google-research/tree/master/goemotions

[2] Kim, Y. (2014). "Convolutional Neural Networks for Sentence Classification."
    Proceedings of the 2014 Conference on Empirical Methods in Natural Language Processing (EMNLP).
    https://aclanthology.org/D14-1181/

[3] Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., Kaiser, L.,
    & Polosukhin, I. (2017). "Attention is all you need." Advances in neural information processing systems.
    https://proceedings.neurips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf

[4] Lin, T. Y., Goyal, P., Girshick, R., He, K., & Dollár, P. (2017).
    "Focal loss for dense object detection."
    Proceedings of the IEEE international conference on computer vision.
    https://arxiv.org/abs/1708.02002

[5] Lipton, Z. C., Elkan, C., & Naryanaswamy, B. (2014).
    "Optimal thresholding of classifiers to maximize F1 measure."
    Joint European Conference on Machine Learning and Knowledge Discovery in Databases.
"""

## preparing the environment


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, Conv1D
from tensorflow.keras.layers import LSTM, Bidirectional, Dropout, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import random
import os
from tensorflow.keras import layers

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.19.0


## Loading dataset
Getting the GoEmotions dataset from github

In [4]:
import pandas as pd
#Download the goemotions dataset
!gsutil cp -r gs://gresearch/goemotions/data/full_dataset/ .

# Load the CSV files
df1 = pd.read_csv("full_dataset/goemotions_1.csv")
df2 = pd.read_csv("full_dataset/goemotions_2.csv")
df3 = pd.read_csv("full_dataset/goemotions_3.csv")

# Concatenate them into one dataframe
df = pd.concat([df1, df2, df3], ignore_index=True)

# To reduce the runtime we use a random sample of 50.000 datapoints
df_small = df.sample(n=50000, random_state=42)

Copying gs://gresearch/goemotions/data/full_dataset/goemotions_1.csv...
/ [0 files][    0.0 B/ 13.5 MiB]                                                
/ [0 files][320.0 KiB/ 13.5 MiB]                                                
-
\
\ [1 files][ 13.5 MiB/ 13.5 MiB]                                                
Copying gs://gresearch/goemotions/data/full_dataset/goemotions_2.csv...
\ [1 files][ 13.5 MiB/ 27.0 MiB]                                                
\ [1 files][ 13.8 MiB/ 27.0 MiB]                                                
|
/
/ [2 files][ 27.0 MiB/ 27.0 MiB]                                                
Copying gs://gresearch/goemotions/data/full_dataset/goemotions_3.csv...
/ [2 files][ 27.0 MiB/ 40.8 MiB]                                                
/ [2 files][ 27.4 MiB/ 40.8 MiB]                                                
-
\
\ [3 files][ 40.8 MiB/ 40.8 MiB]                                                

Operation completed over 3 objects/40.8 Mi

## EDA
We can see that the dataset is unbalanced, and some emotions are not represented a lot

In [5]:
# Display basic info about the dataset
print(f"Dataset shape: {df_small.shape}")

# Identify emotion columns
emotion_columns = df_small.columns[df_small.columns.get_loc('example_very_unclear') + 1:].tolist()
print(f"Number of emotion labels: {len(emotion_columns)}")
print(f"Emotion labels: {emotion_columns}")

# Check label distribution
label_counts = df_small[emotion_columns].sum().sort_values(ascending=False)
print("\nEmotion distribution:")
print(label_counts)

# Visualize label distribution
plt.figure(figsize=(14, 8))
sns.barplot(x=label_counts.index, y=label_counts.values)
plt.xticks(rotation=90)
plt.title('Emotion Distribution in Dataset')
plt.tight_layout()
plt.savefig('emotion_distribution.png')
plt.close()

# Check for texts with no emotion labels
no_emotion = (df_small[emotion_columns].sum(axis=1) == 0).sum()
print(f"\nTexts with no emotion labels: {no_emotion} ({no_emotion/len(df_small)*100:.2f}%)")

Dataset shape: (50000, 37)
Number of emotion labels: 28
Emotion labels: ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']

Emotion distribution:
neutral           13170
admiration         4148
approval           4042
annoyance          3223
gratitude          2738
disapproval        2734
curiosity          2286
amusement          2241
optimism           2118
realization        2037
disappointment     1998
love               1935
anger              1934
joy                1866
confusion          1728
sadness            1549
caring             1352
excitement         1314
disgust            1268
surprise           1267
desire              910
fear                733
remorse             608
embarrassment      

## Preprocessing data
Deviding the data in a test and training set. \\
Text preprocessing with tokenization and padding.We use tokenization to convert raw text into numbers, and padding to make all input sequences the same length — because neural networks don’t understand text and can’t process variable-length input.
Based on techniques described in the Keras documentation:
https://keras.io/api/preprocessing/text/


In [6]:
# Data Preprocessing
# Define features and target
X = df_small['text']
y = df_small[emotion_columns].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text data
max_words = 10000  # Max vocabulary size
max_len = 100  # Max sequence length

tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>') #any word that exceeds the worklimit will be set by that oov token.
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure uniform length
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

print(f"Vocabulary Size: {len(tokenizer.word_index)}")
print(f"Padded Training Data Shape: {X_train_pad.shape}")


Vocabulary Size: 23655
Padded Training Data Shape: (40000, 100)


## Handling imbalance
"""
Calculate class weights to handle imbalance.
This helps address the class imbalance issue common in emotion datasets.
Approach adapted from scikit-learn's class_weight='balanced' principle.
"""

In [7]:
class_counts = np.sum(y_train, axis=0)
total_samples = len(y_train)
class_weights = {}
for i in range(len(emotion_columns)):
    # Calculate weight as inverse of class frequency, normalized
    weight = total_samples / (len(emotion_columns) * class_counts[i])
    class_weights[i] = min(weight, 10.0)  # Cap the weight to prevent extreme values

print("Class weights:", class_weights)

Class weights: {0: np.float64(0.4293872643737387), 1: np.float64(0.7963051441312311), 2: np.float64(0.934317481080071), 3: np.float64(0.5597850425436632), 4: np.float64(0.4411894467484338), 5: np.float64(1.3190871916633689), 6: np.float64(1.0299721907508497), 7: np.float64(0.7785130400934216), 8: np.float64(1.9786307874950535), 9: np.float64(0.9070294784580499), 10: np.float64(0.6580246101204185), 11: np.float64(1.3950892857142858), 12: np.float64(2.886002886002886), 13: np.float64(1.3605442176870748), 14: np.float64(2.4461839530332683), 15: np.float64(0.6586313640255549), 16: 10.0, 17: np.float64(0.9523809523809523), 18: np.float64(0.9404683532399135), 19: np.float64(4.214075010535187), 20: np.float64(0.833472245374229), 21: np.float64(5.2521008403361344), 22: np.float64(0.8689607229753216), 23: np.float64(5.668934240362812), 24: np.float64(3.007518796992481), 25: np.float64(1.1690437222352117), 26: np.float64(1.411631846414455), 27: np.float64(0.13487268018990073)}


## Defining loss function
We add an extra penalty for all zero-predictions. Since predicting always 0 for all emotions will result in high metrics but a bad model. we want the model to predict emotions. \\
We are using focal loss because that functions well with an inbalanced dataset.
"""
Custom focal loss with additional penalty for all-zero predictions.
Based on Lin et al., 2017 [4] with our own extension to prevent all-zero predictions.
"""

**TODO check if the penalty can be deleted, since we also have other code that makes sure one emotion is predicted**

In [8]:
def focal_loss_with_penalty(y_true, y_pred, gamma=2.0, alpha=0.25, epsilon=1e-7):
    # Focal loss for multi-label classification
    y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
    cross_entropy = -y_true * tf.math.log(y_pred) - (1 - y_true) * tf.math.log(1 - y_pred)

    p_t = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
    focal_loss = alpha * tf.pow(1 - p_t, gamma) * cross_entropy

    # Add penalty for all-zero predictions
    sum_pred = tf.reduce_sum(y_pred, axis=1)
    all_zero_penalty = 5.0 * tf.exp(-sum_pred)  # Penalty increases as sum approaches zero

    return tf.reduce_mean(focal_loss) + tf.reduce_mean(all_zero_penalty)

## Define F1 score
We use the F1 score as metric for optimizing the models. F1 micro is best for our multi-label emotion detection model because it accounts for class imbalance by aggregating all true positives, false positives, and false negatives across all labels, giving a more realistic measure of overall model performance.

In [9]:

"""
Custom F1 score metric for Keras.
Implements F1 score calculation directly in TensorFlow for use during training.
"""
def f1_metric(y_true, y_pred):
    y_pred_binary = tf.cast(tf.greater(y_pred, 0.5), tf.float32)

    true_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 1), tf.equal(y_pred_binary, 1)), tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(y_pred_binary, 1)), tf.float32))
    false_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 1), tf.equal(y_pred_binary, 0)), tf.float32))

    precision = true_positives / (true_positives + false_positives + tf.keras.backend.epsilon())
    recall = true_positives / (true_positives + false_negatives + tf.keras.backend.epsilon())

    f1 = 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())
    return f1

## prediction function
Makes sure at least one emotion is predicted, by predicting the one with the highest probability.
Based on the principles discussed in Lipton et al., 2014 [5] with our adaptation
for multi-label classification.

In [10]:
"""
Threshold-based prediction function that ensures at least one emotion is predicted.
Addresses the requirement to prevent all-zero predictions.
Based on the principles discussed in Lipton et al., 2014 [5] with our adaptation
for multi-label classification.
"""
def predict_with_threshold(model, X, threshold=0.5):
    y_pred_proba = model.predict(X)
    y_pred = (y_pred_proba >= threshold).astype(int)

    # If any row has all zeros, set the highest probability class to 1
    zero_rows = np.where(np.sum(y_pred, axis=1) == 0)[0]
    for row in zero_rows:
        max_prob_idx = np.argmax(y_pred_proba[row])
        y_pred[row, max_prob_idx] = 1

    return y_pred

## LSTM model

"""
Model 1: Bidirectional LSTM for text classification.
Based on architecture described in:
Schuster, M., & Paliwal, K. K. (1997). "Bidirectional recurrent neural networks."
IEEE transactions on Signal Processing, 45(11), 2673-2681.
"""

**TODO: we can still check if optimizing the parameters helps. But the Transformer model is the best anyways.**

In [11]:
def build_lstm_model(vocab_size, embedding_dim=100, lstm_units=128, dropout_rate=0.5):
    inputs = Input(shape=(max_len,))
    x = Embedding(vocab_size, embedding_dim)(inputs)
    x = Bidirectional(LSTM(lstm_units, return_sequences=True))(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(len(emotion_columns), activation='sigmoid')(x)

    model = Model(inputs, outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss=focal_loss_with_penalty,
        metrics=['accuracy', f1_metric]
    )
    return model

## CNN model
"""
Model 2: 1D CNN for text classification.
Implements a multi-filter CNN architecture inspired by:
Kim, Y. (2014). "Convolutional Neural Networks for Sentence Classification." [2]
"""

***TODO: optimize parameters, but transformer model is going to be better anyway***

In [12]:
def build_cnn_model(vocab_size, embedding_dim=100, filters=128, kernel_size=5, dropout_rate=0.5):
    inputs = Input(shape=(max_len,))
    x = Embedding(vocab_size, embedding_dim)(inputs)

    # Multiple conv layers with different kernel sizes
    conv1 = Conv1D(filters, 3, padding='same', activation='relu')(x)
    conv2 = Conv1D(filters, 4, padding='same', activation='relu')(x)
    conv3 = Conv1D(filters, 5, padding='same', activation='relu')(x)

    # Pool each conv layer separately
    pool1 = GlobalMaxPooling1D()(conv1)
    pool2 = GlobalMaxPooling1D()(conv2)
    pool3 = GlobalMaxPooling1D()(conv3)

    # Concatenate the pooled features
    concat = tf.keras.layers.concatenate([pool1, pool2, pool3])
    x = Dense(256, activation='relu')(concat)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(len(emotion_columns), activation='sigmoid')(x)

    model = Model(inputs, outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss=focal_loss_with_penalty,
        metrics=['accuracy', f1_metric]
    )
    return model

## Transformer model
"""
Model 3: Transformer-based model for text classification.
Simplified implementation inspired by:
Vaswani, A., et al. (2017). "Attention is all you need." [3]
"""

We also added positional embedding, since that gives the transformer model information about the position of the words.
the original method introduced in the "Attention Is All You Need" paper by Vaswani et al. (2017), which uses sine and cosine functions of varying frequencies to encode positional information. This approach allows the Transformer model to capture the order of words in a sequence, which is crucial since Transformers lack inherent sequential processing capabilities. \\
Source of code for positional encoding is from Miguel. \\
Since the positional embedding gave us worse metrics in the results we added learning positional embedding. Where just like the normal embedding layer the parameters are learned and optimized. \\
***TODO: optimize parameters***

In [13]:
#normal positional encoding
def get_positional_encoding(seq_length, d_model):
    # Calculate positional encoding
    positions = np.arange(seq_length)[:, np.newaxis]
    depths = np.arange(d_model)[np.newaxis, :] // 2 * 2  # Integer division

    # Create angle rates
    angle_rates = 1 / np.power(10000, (2 * (depths // 2)) / np.float32(d_model))
    angle_rads = positions * angle_rates

    # Apply sin/cos to even/odd indices
    pos_encoding = np.zeros(angle_rads.shape)
    pos_encoding[:, 0::2] = np.sin(angle_rads[:, 0::2])
    pos_encoding[:, 1::2] = np.cos(angle_rads[:, 1::2])

    return tf.cast(pos_encoding, dtype=tf.float32)

In [14]:
#working model without positional embedding
def build_transformer_model(vocab_size, embedding_dim=100, num_heads=8, ff_dim=128, dropout_rate=0.1):
    inputs = Input(shape=(max_len,))
    embedding_layer = Embedding(vocab_size, embedding_dim)(inputs)

    # Transformer block
    transformer_block = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embedding_dim // num_heads
    )(embedding_layer, embedding_layer)

    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(transformer_block + embedding_layer)

    # Feed Forward
    ff = Dense(ff_dim, activation='relu')(x)
    ff = Dense(embedding_dim)(ff)

    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x + ff)
    x = GlobalMaxPooling1D()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(len(emotion_columns), activation='sigmoid')(x)

    model = Model(inputs, outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss=focal_loss_with_penalty,
        metrics=['accuracy', f1_metric]
    )
    return model

In [15]:
#working model with positional embedding
def build_transformer_model(vocab_size, embedding_dim=100, num_heads=8, ff_dim=128, dropout_rate=0.1):
    inputs = Input(shape=(max_len,))
    embedding_layer = Embedding(vocab_size, embedding_dim)(inputs)

    # Embedding layer for positional encoding
    pos_encoding = get_positional_encoding(max_len,  embedding_dim)
    pos_encoding = tf.keras.backend.constant(pos_encoding)

    embedded_with_pos = embedding_layer + pos_encoding

    # Transformer block
    transformer_block = layers.MultiHeadAttention(
        num_heads=2, key_dim=64
    )(embedded_with_pos, embedded_with_pos)

    norm_transformer_block = layers.LayerNormalization(epsilon=1e-6)(transformer_block + embedded_with_pos)

    # Feed Forward
    ff = Dense(ff_dim, activation='relu')(norm_transformer_block)
    ff = Dense(embedding_dim)(ff)

    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(norm_transformer_block + ff)
    x = GlobalMaxPooling1D()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(len(emotion_columns), activation='sigmoid')(x)

    model = Model(inputs, outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss=focal_loss_with_penalty,
        metrics=['accuracy', f1_metric]
    )
    return model

Learning positional embedding

In [16]:
class LearnablePositionalEncoding(layers.Layer):
    def __init__(self, maxlen, embedding_dim):
        super().__init__()
        self.pos_embedding = self.add_weight(
            shape=(maxlen, embedding_dim),
            initializer='random_normal',
            trainable=True,
            name="learnable_pos_embedding"
        )

    def call(self, x):
        # x shape: (batch_size, sequence_length, embedding_dim)
        seq_len = tf.shape(x)[1]
        return x + self.pos_embedding[tf.newaxis, :seq_len, :]

In [17]:
#model with learnable positional encoding
def build_transformer_model(vocab_size, embedding_dim=100, num_heads=8, ff_dim=128, dropout_rate=0.1):
    inputs = Input(shape=(max_len,))

    # Embedding + positional encoding
    x = Embedding(vocab_size, embedding_dim)(inputs)
    x = LearnablePositionalEncoding(maxlen=max_len, embedding_dim=embedding_dim)(x)

    # Transformer block
    attention_output = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embedding_dim // num_heads
    )(x, x)

    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention_output + x)

    # Feed Forward
    ff = Dense(ff_dim, activation='relu')(x)
    ff = Dense(embedding_dim)(ff)
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x + ff)

    # Classification head
    x = GlobalMaxPooling1D()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(len(emotion_columns), activation='sigmoid')(x)

    model = Model(inputs, outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss=focal_loss_with_penalty,
        metrics=['accuracy', f1_metric]
    )
    return model


## Callbakcs
Introduce early stopping so the model will stop after 3 epochs after eachother do not improve the f1 score. The vest model is saved.

In [18]:
# Set up callbacks
callbacks = [
    EarlyStopping(monitor='val_f1_metric', patience=4, mode='max', restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_f1_metric', mode='max', save_best_only=True)
]


## Training and evaluating the models

***TODO: check parameters like epochs, batchsize***

In [19]:
def train_and_evaluate(model, model_name):
    print(f"\n=== Training {model_name} ===")

    history = model.fit(
        X_train_pad, y_train,
        epochs=10,
        batch_size=32,
        validation_split=0.2,
        callbacks=callbacks,
        class_weight=class_weights,
        verbose=1
    )

    # Evaluate on test set
    y_pred = predict_with_threshold(model, X_test_pad, threshold=0.5)

    # Calculate F1 scores
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    micro_f1 = f1_score(y_test, y_pred, average='micro')
    weighted_f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"\n{model_name} Results:")
    print(f"Macro F1 Score: {macro_f1:.4f}")
    print(f"Micro F1 Score: {micro_f1:.4f}")
    print(f"Weighted F1 Score: {weighted_f1:.4f}")

    # Check if any all-zero predictions remain
    zero_preds = (np.sum(y_pred, axis=1) == 0).sum()
    print(f"Texts with no emotion predictions: {zero_preds} ({zero_preds/len(y_test)*100:.2f}%)")

    # Plot training history
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'{model_name} - Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['f1_metric'], label='Train F1')
    plt.plot(history.history['val_f1_metric'], label='Val F1')
    plt.title(f'{model_name} - F1 Score')
    plt.legend()

    plt.tight_layout()
    plt.savefig(f'{model_name}_training_history.png')
    plt.close()

    return model, macro_f1, micro_f1, weighted_f1, y_pred

### LSTM model

In [20]:
# Initialize and train LSTM model
print("\nBuilding LSTM model...")
lstm_model = build_lstm_model(vocab_size=min(len(tokenizer.word_index) + 1, max_words))
lstm_model.summary()
lstm_model, lstm_macro_f1, lstm_micro_f1, lstm_weighted_f1, lstm_preds = train_and_evaluate(lstm_model, "LSTM")


Building LSTM model...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100, 256)       │       234,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 256)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 28)             │         7,196 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,307,484 (4.99 MB)

 Trainable params: 1,307,484 (4.99 MB)

...

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 139s 130ms/step - accuracy: 0.2431 - f1_metric: 0.0742 - loss: 0.0119 - val_accuracy: 0.3685 - val_f1_metric: 0.2309 - val_loss: 0.0132
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.3681 - f1_metric: 0.2235 - loss: 0.0100

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 120s 120ms/step - accuracy: 0.3681 - f1_metric: 0.2235 - loss: 0.0100 - val_accuracy: 0.3882 - val_f1_metric: 0.3036 - val_loss: 0.0129
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.3937 - f1_metric: 0.3040 - loss: 0.0097

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 112s 112ms/step - accuracy: 0.3937 - f1_metric: 0.3040 - loss: 0.0097 - val_accuracy: 0.3901 - val_f1_metric: 0.3259 - val_loss: 0.0129
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.4242 - f1_metric: 0.3578 - loss: 0.0094

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 115s 115ms/step - accuracy: 0.4242 - f1_metric: 0.3578 - loss: 0.0094 - val_accuracy: 0.3831 - val_f1_metric: 0.3318 - val_loss: 0.0129
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.4599 - f1_metric: 0.4079 - loss: 0.0091

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 130s 129ms/step - accuracy: 0.4599 - f1_metric: 0.4079 - loss: 0.0091 - val_accuracy: 0.3691 - val_f1_metric: 0.3369 - val_loss: 0.0131
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 125s 125ms/step - accuracy: 0.5018 - f1_metric: 0.4609 - loss: 0.0088 - val_accuracy: 0.3577 - val_f1_metric: 0.3335 - val_loss: 0.0134
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 117s 117ms/step - accuracy: 0.5393 - f1_metric: 0.5101 - loss: 0.0086 - val_accuracy: 0.3471 - val_f1_metric: 0.3309 - val_loss: 0.0138
Epoch 8/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 119s 119ms/step - accuracy: 0.5740 - f1_metric: 0.5551 - loss: 0.0083 - val_accuracy: 0.3338 - val_f1_metric: 0.3263 - val_loss: 0.0142
Epoch 9/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 121s 121ms/step - accuracy: 0.5989 - f1_metric: 0.5923 - loss: 0.0081 - val_accuracy: 0.3371 - val_f1_metric: 0.3280 - val_loss: 0.0151
313/313 ━━━━━━━━━━━━━━━━━━━━ 18s 57ms/step

LSTM Results:
Macro F1 Score: 0.2409
Micro F1 Score: 0.3773
Weighted F1 Score

### CNN model

In [21]:
# Initialize and train CNN model
print("\nBuilding CNN model...")
cnn_model = build_cnn_model(vocab_size=min(len(tokenizer.word_index) + 1, max_words))
cnn_model.summary()
cnn_model, cnn_macro_f1, cnn_micro_f1, cnn_weighted_f1, cnn_preds = train_and_evaluate(cnn_model, "CNN")



Building CNN model...


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 100, 100)  │  1,000,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 100, 128)  │     38,528 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 100, 128)  │     51,328 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 100, 128)  │     64,128 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d[0][0]      │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_1[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_2[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 384)       │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
│                     │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │     98,560 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 28)        │      7,196 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,259,740 (4.81 MB)

 Trainable params: 1,259,740 (4.81 MB)

 Non-trainable params: 0 (0.00 B)


=== Training CNN ===
Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 35s 30ms/step - accuracy: 0.2437 - f1_metric: 0.0803 - loss: 0.0121 - val_accuracy: 0.3756 - val_f1_metric: 0.2284 - val_loss: 0.0132
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - accuracy: 0.3706 - f1_metric: 0.2286 - loss: 0.0101 - val_accuracy: 0.3891 - val_f1_metric: 0.3090 - val_loss: 0.0129
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 24s 24ms/step - accuracy: 0.4010 - f1_metric: 0.3198 - loss: 0.0097 - val_accuracy: 0.3918 - val_f1_metric: 0.3284 - val_loss: 0.0129
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 42s 42ms/step - accuracy: 0.4485 - f1_metric: 0.3879 - loss: 0.0093 - val_accuracy: 0.3751 - val_f1_metric: 0.3273 - val_loss: 0.0131
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 75s 34ms/step - accuracy: 0.4997 - f1_metric: 0.4621 - loss: 0.0089 - val_accuracy: 0.3646 - val_f1_metric: 0.3342 - val_loss: 0.0134
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 41s 40ms/step - accuracy: 0.5598 - f1_metric: 0.5330 - l

### Transformer model

In [22]:
# Initialize and train Transformer model
print("\nBuilding Transformer model...")
transformer_model = build_transformer_model(vocab_size=min(len(tokenizer.word_index) + 1, max_words))
transformer_model.summary()
transformer_model, transformer_macro_f1, transformer_micro_f1, transformer_weighted_f1, transformer_preds = train_and_evaluate(transformer_model, "Transformer")



Building Transformer model...



Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 100, 100)  │  1,000,000 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ learnable_position… │ (None, 100, 100)  │     10,000 │ embedding_2[0][0] │
│ (LearnablePosition… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 100, 100)  │     38,788 │ learnable_positi… │
│ (MultiHeadAttentio… │                   │            │ learnable_positi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 100, 100)  │          0 │ multi_head_atten… │
│                     │                   │            │ learnable_positi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 100, 100)  │        200 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 100, 128)  │     12,928 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 100, 100)  │     12,900 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 100, 100)  │          0 │ layer_normalizat… │
│                     │                   │            │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 100, 100)  │        200 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 100)       │          0 │ layer_normalizat… │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 100)       │          0 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 256)       │     25,856 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 256)       │          0 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 28)        │      7,196 │ dropout_4[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,108,068 (4.23 MB)

 Trainable params: 1,108,068 (4.23 MB)

 Non-trainable params: 0 (0.00 B)


=== Training Transformer ===
Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 50s 48ms/step - accuracy: 0.2231 - f1_metric: 0.0530 - loss: 0.0128 - val_accuracy: 0.3501 - val_f1_metric: 0.1559 - val_loss: 0.0135
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 25s 25ms/step - accuracy: 0.3589 - f1_metric: 0.1860 - loss: 0.0101 - val_accuracy: 0.3865 - val_f1_metric: 0.3288 - val_loss: 0.0131
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 29s 29ms/step - accuracy: 0.3928 - f1_metric: 0.2897 - loss: 0.0097 - val_accuracy: 0.3919 - val_f1_metric: 0.3251 - val_loss: 0.0130
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 57s 57ms/step - accuracy: 0.4163 - f1_metric: 0.3429 - loss: 0.0095 - val_accuracy: 0.3837 - val_f1_metric: 0.3308 - val_loss: 0.0131
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 68s 44ms/step - accuracy: 0.4484 - f1_metric: 0.3953 - loss: 0.0092 - val_accuracy: 0.3652 - val_f1_metric: 0.3319 - val_loss: 0.0132
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 35s 35ms/step - accuracy: 0.4865 - f1_metric: 0.

## Comparing the models
We use the micro F1 score to compare the models

In [23]:
# Compare models
models_comparison = {
    'Model': ['BiLSTM', 'CNN', 'Transformer'],
    'Macro F1': [lstm_macro_f1, cnn_macro_f1, transformer_macro_f1],
    'Micro F1': [lstm_micro_f1, cnn_micro_f1, transformer_micro_f1],
    'Weighted F1': [lstm_weighted_f1, cnn_weighted_f1, transformer_weighted_f1]
}

comparison_df = pd.DataFrame(models_comparison)
print("\nModel Comparison:")
print(comparison_df)

# Identify best model
best_idx = np.argmax([lstm_micro_f1, cnn_micro_f1, transformer_micro_f1])
best_models = [lstm_model, cnn_model, transformer_model]
best_model = best_models[best_idx]
best_model_name = models_comparison['Model'][best_idx]

print(f"\nBest model: {best_model_name} with Micro F1: {models_comparison['Micro F1'][best_idx]:.4f}")


Model Comparison:
         Model  Macro F1  Micro F1  Weighted F1
0       BiLSTM  0.240904  0.377341     0.327515
1          CNN  0.238546  0.371019     0.324115
2  Transformer  0.250132  0.363032     0.323398

Best model: BiLSTM with Micro F1: 0.3773


## Optimize threshold
For the best model found above we optimize the threshold to maximize the f1 score. \\
Based on the approach described in:
Lipton, Z. C., Elkan, C., & Naryanaswamy, B. (2014) [5] \\
We also check the f1 score per emotion.

In [24]:
def optimize_threshold(model, X, y_true, thresholds=None):
    if thresholds is None:
        thresholds = np.arange(0.2, 0.8, 0.05)

    results = []

    for threshold in thresholds:
        y_pred = predict_with_threshold(model, X, threshold=threshold)
        micro_f1 = f1_score(y_true, y_pred, average='micro')
        results.append((threshold, micro_f1))
        print(f"Threshold: {threshold:.2f}, Micro F1: {micro_f1:.4f}")

    best_threshold, best_f1 = max(results, key=lambda x: x[1])
    print(f"\nBest threshold: {best_threshold:.2f}")
    print(f"Best F1 score: {best_f1:.4f}")

    # Plot thresholds vs F1 scores
    thresholds, f1_scores = zip(*results)
    plt.figure(figsize=(10, 6))
    plt.plot(thresholds, f1_scores, marker='o')
    plt.xlabel('Threshold')
    plt.ylabel('Micro F1 Score')
    plt.title('Threshold Optimization')
    plt.grid(True)
    plt.savefig('threshold_optimization.png')
    plt.close()

    return best_threshold

"""
Threshold Optimization and Final Evaluation
"""
# Optimize threshold for best model
print("\nOptimizing threshold for best model...")
best_threshold = optimize_threshold(best_model, X_test_pad, y_test)

# Get final predictions with optimized threshold
final_preds = predict_with_threshold(best_model, X_test_pad, threshold=best_threshold)
final_macro_f1 = f1_score(y_test, final_preds, average='macro')
final_micro_f1 = f1_score(y_test, final_preds, average='micro')
final_weighted_f1 = f1_score(y_test, final_preds, average='weighted')
per_class_f1 = f1_score(y_test, final_preds, average=None)

print("\nFinal Model Results with Optimized Threshold:")
print(f"Macro F1 Score: {final_macro_f1:.4f}")
print(f"Micro F1 Score: {final_micro_f1:.4f}")
print(f"Weighted F1 Score: {final_weighted_f1:.4f}")

# Print per-class F1 scores
print("\nF1 Score by emotion:")
for i, emotion in enumerate(emotion_columns):
    print(f"{emotion}: {per_class_f1[i]:.4f}")

# Visualize per-class F1 scores
plt.figure(figsize=(14, 8))
emotion_f1_dict = dict(zip(emotion_columns, per_class_f1))
sorted_emotion_f1 = {k: v for k, v in sorted(emotion_f1_dict.items(), key=lambda item: item[1], reverse=True)}
sns.barplot(x=list(sorted_emotion_f1.keys()), y=list(sorted_emotion_f1.values()))
plt.xticks(rotation=90)
plt.title('F1 Score by Emotion')
plt.ylim(0, 1)
plt.tight_layout()
plt.savefig('emotion_f1_scores.png')
plt.close()


Optimizing threshold for best model...
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step
Threshold: 0.20, Micro F1: 0.0813
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step
Threshold: 0.25, Micro F1: 0.1046
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step
Threshold: 0.30, Micro F1: 0.2350
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step
Threshold: 0.35, Micro F1: 0.3371
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step
Threshold: 0.40, Micro F1: 0.3884
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step
Threshold: 0.45, Micro F1: 0.3881
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step
Threshold: 0.50, Micro F1: 0.3773
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step
Threshold: 0.55, Micro F1: 0.3731
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step
Threshold: 0.60, Micro F1: 0.3729
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step
Threshold: 0.65, Micro F1: 0.3726
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step
Threshold: 0.70, Micro F1: 0.3725
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step
Threshold: 0.75, Micro F1: 0.3725
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step
Thresh

# Speech to text experiment for the best model

In [25]:
import os
import whisper
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from sklearn.metrics import accuracy_score  # For calculating accuracy

In [28]:
# Initialize the Whisper model
whisper_model = whisper.load_model("base")  # Use your desired model: "base", "small", "medium", or "large"

# Load your sentiment analysis model (make sure it's trained and saved previously)
# Example:
# model = keras.models.load_model('path_to_your_sentiment_model.h5')

# Path to the audio files folder
audio_folder_path = 'Audio_Files/'
emotion_data = pd.read_csv('labels_2.csv')

def get_actual_emotion(filename):
    # Look for the corresponding row for the filename in the CSV file
    row = emotion_data[emotion_data['filename'] == filename]
    if not row.empty:
        return row.iloc[0]['emotion']  # Return the emotion value from the third column
    return None  # In case the file is not found


def predict_emotions(texts, model=best_model, emotion_cols=emotion_columns, tokenizer=tokenizer, threshold=best_threshold):
    # Preprocess the texts
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

    # Get predictions
    y_pred_proba = model.predict(padded_sequences)
    y_pred = (y_pred_proba >= threshold).astype(int)


    # Ensure at least one emotion is predicted for each text
    zero_rows = np.where(np.sum(y_pred, axis=1) == 0)[0]
    for row in zero_rows:
        max_prob_idx = np.argmax(y_pred_proba[row])
        y_pred[row, max_prob_idx] = 1

    # Create results as a DataFrame
    results = pd.DataFrame(y_pred, columns=emotion_cols)
    results.insert(0, 'text', texts)

    return results

# Function to process each audio file and predict emotion using LSTM model
def process_audio(audio_path, audio_file):
    try:
        # Transcribe the audio file using Whisper
        result = whisper_model.transcribe(audio_path)
        transcribed_text = result['text']
        print(f"Transcription for {audio_file}: {transcribed_text}")

        predictions = predict_emotions([transcribed_text])
        predicted_emotions_list = predictions.loc[:, (predictions != 0).any(axis=0)]
        #get columnnames of predicted_emotions
        predicted_emotion = predicted_emotions_list.columns[1:]

        # Get the actual emotion for comparison
        actual_emotion = get_actual_emotion(audio_file)

        # Print the predicted emotion
        print(f"Predicted Emotion for {audio_file}: {predicted_emotion}")

        # Return the transcription, predicted emotion, and actual emotion
        return transcribed_text, list(predicted_emotion), actual_emotion
    except Exception as e:
        print(f"Error with Whisper transcription for {audio_path}: {e}")
        return None, None, None

# List to store predicted and actual emotions for evaluation
predicted_emotions = []
actual_emotions = []

In [29]:
# Loop through all audio files in the folder and process them
for audio_file in os.listdir(audio_folder_path):
    if audio_file.endswith(('.mp4','.mp3', '.m4a', '.wav', '.flac')):  # Check for relevant audio file extensions
        audio_path = os.path.join(audio_folder_path, audio_file)
        transcribed_text, predicted_emotion, actual_emotion = process_audio(audio_path, audio_file)

        if transcribed_text and predicted_emotion and actual_emotion:
            print(f"Audio File: {audio_file} -> Predicted Emotion: {predicted_emotion}, Actual Emotion: {actual_emotion}")
            # Append the results for accuracy calculation
            predicted_emotions.append(predicted_emotion)
            actual_emotions.append(actual_emotion)

# Ensure no missing values before calculating accuracy
if predicted_emotions and actual_emotions:
    try:
        # Compute accuracy score
        accuracy = accuracy_score(actual_emotions, predicted_emotions)
        print(f"Model Accuracy: {accuracy * 100:.2f}%")
    except ValueError as e:
        print(f"Error in accuracy calculation: {e}")
else:
    print("Error: Missing predictions or actual emotions, cannot calculate accuracy.")

c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription for audio1.m4a:  That was a fantastic presentation. I'm really impressed.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Predicted Emotion for audio1.m4a: Index(['admiration'], dtype='object')
Audio File: audio1.m4a -> Predicted Emotion: ['admiration'], Actual Emotion: approval


c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription for audio10.m4a:  After watching the performance, I couldn't help but admire the level of skill and dedication.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted Emotion for audio10.m4a: Index(['admiration', 'neutral'], dtype='object')
Audio File: audio10.m4a -> Predicted Emotion: ['admiration', 'neutral'], Actual Emotion: admiration


c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

Transcription for audio97.m4a:  If only I had taken a moment to think it through, maybe it wouldn't feel this way right now.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Predicted Emotion for audio97.m4a: Index(['neutral'], dtype='object')
Audio File: audio97.m4a -> Predicted Emotion: ['neutral'], Actual Emotion: remorse


c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription for audio98.m4a:  As I walked through the empty house, the realization that it was all over hit me hard. The silence was felt so heavy.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Predicted Emotion for audio98.m4a: Index(['neutral'], dtype='object')
Audio File: audio98.m4a -> Predicted Emotion: ['neutral'], Actual Emotion: sadness


c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription for audio99.m4a:  It's hard to put into words but seeing everything change felt like a dip sadness that I can quite to explain.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
Predicted Emotion for audio99.m4a: Index(['disappointment', 'sadness'], dtype='object')
Audio File: audio99.m4a -> Predicted Emotion: ['disappointment', 'sadness'], Actual Emotion: sadness
Error in accuracy calculation: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.


In [30]:
# To get the accuracy

correct = 0
for pred, actual in zip(predicted_emotions, actual_emotions):
    if actual in pred:
        correct += 1

accuracy = correct / len(actual_emotions)
print(f"Accuracy (actual in predicted): {accuracy*100:.2f}%")

Accuracy (actual in predicted): 23.41%


# Now run the model for the original script 

In [31]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# Load your trained components
# (Make sure these are already defined / loaded in your script)
# best_model, tokenizer, emotion_columns, max_len, best_threshold

# Load the CSV with text and ground‐truth emotions
emotion_data = pd.read_csv('labels_2.csv')

def predict_emotions(texts,
                     model=best_model,
                     tokenizer=tokenizer,
                     emotion_cols=emotion_columns,
                     threshold=best_threshold,
                     max_len=max_len):
    # Tokenize & pad
    sequences = tokenizer.texts_to_sequences(texts)
    padded = pad_sequences(sequences, maxlen=max_len,
                           padding='post', truncating='post')

    # Get probability predictions
    y_proba = model.predict(padded)
    y_pred = (y_proba >= threshold).astype(int)

    # Guarantee at least one emotion per text
    zero_rows = np.where(y_pred.sum(axis=1) == 0)[0]
    for i in zero_rows:
        y_pred[i, y_proba[i].argmax()] = 1

    # Build DataFrame
    df = pd.DataFrame(y_pred, columns=emotion_cols)
    df.insert(0, 'text', texts)
    return df

def process_row(raw_text, actual_emotion):
    print(f"Text: {raw_text}")
    df_pred = predict_emotions([raw_text])
    # get list of predicted emotion labels
    preds = df_pred.loc[0, emotion_columns]
    predicted_list = preds[preds == 1].index.tolist()

    print(f"Predicted: {predicted_list}, Actual: {actual_emotion}\n")
    return predicted_list, actual_emotion

predicted_emotions = []
actual_emotions = []

# Iterate rows in your CSV
for idx, row in emotion_data.iterrows():
    text = row['text']
    actual = row['emotion']  # assumes this is a single label per row
    pred_list, act = process_row(text, actual)

    if pred_list and act is not None:
        predicted_emotions.append(pred_list)
        actual_emotions.append(act)

# For multi‐label, exact‐match accuracy:
matches = [1 if actual_emotions[i] in predicted_emotions[i] else 0
           for i in range(len(actual_emotions))]
accuracy = sum(matches) / len(matches)
print(f"Exact‐match Accuracy: {accuracy * 100:.2f}%")

Text: That was a fantastic presentation. I'm really impressed. 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Predicted: ['admiration'], Actual: approval

Text: I cannot believe this is happening again. It is so frustrating.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Predicted: ['anger', 'annoyance', 'disapproval'], Actual: annoyance

Text: I really don't think this is the right decision.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Predicted: ['disapproval', 'neutral'], Actual: disapproval 

Text: Thank you so much for your help. I truly appreciate it.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted: ['admiration', 'gratitude'], Actual: gratitude

Text: I wonder how that works. I'd love to know more about it.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted: ['excitement', 'surprise'], Actual: curiosity

Text: Oh, I get it now. That was why everything wasn't adding up.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted: ['curiosity', 'neutral'], Actual: realization

Text: I'm sure things will get better soon

# Audio Streaming


In [47]:
def predict_emotions(texts, model=best_model, emotion_cols=emotion_columns, tokenizer=tokenizer, threshold=best_threshold):
    # Preprocess the texts
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

    # Get predictions
    y_pred_proba = model.predict(padded_sequences)
    y_pred = (y_pred_proba >= threshold).astype(int)

    #print(y_pred_proba)
    # Ensure at least one emotion is predicted for each text
    zero_rows = np.where(np.sum(y_pred, axis=1) == 0)[0]
    for row in zero_rows:
        max_prob_idx = np.argmax(y_pred_proba[row])
        y_pred[row, max_prob_idx] = 1

    # Create results as a DataFrame
    results = pd.DataFrame(y_pred, columns=emotion_cols)
    results.insert(0, 'text', texts)

    return results

In [ ]:
import whisper
import numpy as np
import queue
import threading
import pyaudio
import time
from tensorflow.keras.preprocessing.sequence import pad_sequences


CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 3

p = pyaudio.PyAudio()
audio_queue = queue.Queue()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

def process_audio():
    while True:
        audio_data = audio_queue.get()
        # convert to float32 in [-1,1]
        audio_np = np.frombuffer(audio_data, dtype=np.int16).astype(np.float32) / 32768.0

        # Whisper transcription
        result = whisper_model.transcribe(audio_np, fp16=False)
        text = result["text"].strip()
        if not text:
            continue

        print(f"\nTranscribed: {text}")

        # Sentiment
        sentiment = predict_emotions(text, model=best_model, emotion_cols=emotion_columns, tokenizer=tokenizer, threshold=best_threshold)
        predicted_emotions = sentiment.loc[:, (sentiment != 0).any(axis=0)]
        #get columnnames of predicted_emotions
        print(predicted_emotions.columns[1:])
    


# start background thread
threading.Thread(target=process_audio, daemon=True).start()

try:
    print("* Recording started — press Ctrl+C to stop")
    while True:
        frames = []
        for _ in range(int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK, exception_on_overflow=False)
            frames.append(data)
        audio_queue.put(b"".join(frames))

except KeyboardInterrupt:
    print("\n* Recording stopped")

finally:
    stream.stop_stream()
    stream.close()
    p.terminate()

* Recording started — press Ctrl+C to stop

Transcribed: I'm glad the project is done.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Index(['admiration', 'approval', 'disapproval', 'neutral'], dtype='object')

* Recording stopped



Transcribed: Oh good. Yeah.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Index(['neutral'], dtype='object')
